- 18846개의 뉴스 문서를 20개의 뉴스 그룹 카테고리로 분류

텍스트 정규화 -> 피쳐 벡터화 -> 머신러닝 학습/예측/평가 -> pipeline 적용 -> GridSearchCV 최적화

# 데이터 로드

In [1]:
from sklearn.datasets import fetch_20newsgroups

news_data = fetch_20newsgroups(subset='all', random_state=216)

# 데이터 구성 확인

In [5]:
type(news_data) # Bunch 타입임을 확인

sklearn.utils.Bunch

In [7]:
print(news_data.keys())

dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR'])


In [8]:
# DESCR 에서 해당 데이터 셋에 대한 설명
print(news_data.DESCR)

.. _20newsgroups_dataset:

The 20 newsgroups text dataset
------------------------------

The 20 newsgroups dataset comprises around 18000 newsgroups posts on
20 topics split in two subsets: one for training (or development)
and the other one for testing (or for performance evaluation). The split
between the train and test set is based upon a messages posted before
and after a specific date.

This module contains two loaders. The first one,
:func:`sklearn.datasets.fetch_20newsgroups`,
returns a list of the raw texts that can be fed to text feature
extractors such as :class:`sklearn.feature_extraction.text.CountVectorizer`
with custom parameters so as to extract feature vectors.
The second one, :func:`sklearn.datasets.fetch_20newsgroups_vectorized`,
returns ready-to-use features, i.e., it is not necessary to use a feature
extractor.

**Data Set Characteristics:**

    =================   ==========
    Classes                     20
    Samples total            18846
    Dimensionality 

In [10]:
import pandas as pd

print("target 클래스의 이름들 :", news_data.target_names)
print("target 클래스의 값과 분포도\n ", pd.Series(news_data.target).value_counts().sort_index() )

target 클래스의 이름들 : ['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']
target 클래스의 값과 분포도
  0     799
1     973
2     985
3     982
4     963
5     988
6     975
7     990
8     996
9     994
10    999
11    991
12    984
13    990
14    987
15    997
16    910
17    940
18    775
19    628
dtype: int64


In [12]:
print(news_data.data[0])

From: filinuk@staff.dccs.upenn.edu (Geoff Filinuk)
Subject: Wash/Isl OT game
Reply-To: filinuk@staff.dccs.upenn.edu (Geoff Filinuk)
Organization: University of Pennsylvania
Lines: 17
Nntp-Posting-Host: staff.dccs.upenn.edu


	This comes indirectly from Al Morgani who works in the studio
for ESPN hockey.

	The management of ESPN was reluctant to leave the Pitt-NJ game 
(even though the outcome was certain) because of fear of the Wash-Ny game 
going to overtime.  When the Caps scored with :02 left to tie the game, Al said
he heard many people say f**K .!#$%.  
	ESPN is under contractual obligation to show baseball and COULD NOT
broadcast the OT of the hockey game.  Next year, ESPN2 will be introduced so 
baseball fans can watch baseball and hockey fans can watch hockey.

	GET OFF ESPN'S BACK AND STOP POSTING ARTICLES ON THIS SUBJECT!!!
There have been to many f****n complainers about this game that it's making me
sick.

Geoff Filinuk



# 학습, 테스트용 데이터 생성

In [15]:
from sklearn.datasets import fetch_20newsgroups

# subset='train'으로 학습용(Train) 데이터만 추출
# remove=('headers', 'footers', 'quotes')로 실제 텍스트만 추출
train_news = fetch_20newsgroups(subset='train', remove=('header','footers','quotes'), random_state=216)

X_train = train_news.data
y_train = train_news.target

# subset='test'로 테스트 데이터만 추출
test_news = fetch_20newsgroups(subset='test', remove=('header','footers','quotes'), random_state=216)
X_test = test_news.data
y_test = test_news.target

In [16]:
print(type(X_train))
print('학습데이터 크기{0}, 테스트데이터 크기{1}'.format(len(train_news.data), len(test_news.data)))

<class 'list'>
학습데이터 크기11314, 테스트데이터 크기7532


# BOW_CountVectorizer_피쳐 벡타화 변환

주의: 학습 데이터에 대해 fit( )된 CountVectorizer를 이용해서 테스트 데이터를 피처 벡터화 해야함 


테스트 데이터에서 fit()을 수행하게 되면 기존 학습된 모델에서 가지는 feature의 갯수가 달라지기 때문

In [17]:
from sklearn.feature_extraction.text import CountVectorizer

cnt_vect = CountVectorizer()
cnt_vect.fit(X_train)
X_train_cnt_vect = cnt_vect.transform(X_train)

# 학습 데이터로 fit( )된 CountVectorizer를 이용하여 테스트 데이터를 feature extraction 변환 수행
X_test_cnt_vect = cnt_vect.transform(X_test)

In [18]:
# 학습 데이터 Text의 CounterVectorizer Shape
print(X_train_cnt_vect.shape)
print(X_test_cnt_vect.shape)

(11314, 99238)
(7532, 99238)


## 머신러닝 모델 학습/예측/평가

In [20]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# text분석 LogisticRegression 효과적인 편
# LogisticRegression을 이용하여 학습/예측/평가 수행
lr_clf = LogisticRegression()
lr_clf.fit(X_train_cnt_vect, y_train)
pred = lr_clf.predict(X_test_cnt_vect)

print('CountVectorized Logistic Regression의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test, pred)))

CountVectorized Logistic Regression의 예측 정확도는 0.749


E:\anaconda\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


# BOW_TF-IDF 피쳐 변환

In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer

# TF-IDF Vectorization 적용하여 학습 데이터셋과 테스트 데이터 셋 변환
tfidf_vect = TfidfVectorizer()
tfidf_vect.fit(X_train)
X_train_tfidf_vect = tfidf_vect.transform(X_train)
X_test_tfidf_vect = tfidf_vect.transform(X_test)

## 머신러닝 모델 학습/예측/평가

In [22]:
# LogisticRegression을 이용하여 학습/예측/평가 수행.
lr_clf = LogisticRegression()
lr_clf.fit(X_train_tfidf_vect , y_train)
pred = lr_clf.predict(X_test_tfidf_vect)

print('TF-IDF Logistic Regression 의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test ,pred)))

TF-IDF Logistic Regression 의 예측 정확도는 0.782


**TF-IDF의 성능이 더 좋음**

## Stop_words 필터링 추가, ngram=(1,2)로 변경하여 피쳐 벡터화

In [25]:
tfidf_vect = TfidfVectorizer(stop_words='english', ngram_range=(1,2), max_df=300) # 300번 이상 나온 단어는 무시
tfidf_vect.fit(X_train)
X_train_tfidf_vect = tfidf_vect.transform(X_train) # 학습 데이터
X_test_tfidf_vect = tfidf_vect.transform(X_test) # 테스트 데이터

lr_clf = LogisticRegression()
lr_clf.fit(X_train_tfidf_vect, y_train)
pred = lr_clf.predict(X_test_tfidf_vect)

print('TF-IDF Logistic Regression 의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test ,pred)))

TF-IDF Logistic Regression 의 예측 정확도는 0.775


**ngram=(1,1) 기본값으로 변경하여 다시 피쳐 벡터화**

In [26]:
tfidf_vect = TfidfVectorizer(stop_words='english', max_df=300) # 300번 이상 나온 단어는 무시
tfidf_vect.fit(X_train)
X_train_tfidf_vect = tfidf_vect.transform(X_train) # 학습 데이터
X_test_tfidf_vect = tfidf_vect.transform(X_test) # 테스트 데이터

lr_clf = LogisticRegression()
lr_clf.fit(X_train_tfidf_vect, y_train)
pred = lr_clf.predict(X_test_tfidf_vect)

print('TF-IDF Logistic Regression 의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test ,pred)))

TF-IDF Logistic Regression 의 예측 정확도는 0.777


## GridSearchCV로 LR C 하이퍼파라미터 튜닝

C: 규제 계수의 역수

In [27]:
from sklearn.model_selection import GridSearchCV

# 최적 C 값 도출 튜닝 수행
# CV = 3 Fold셋 설정

params = { 'C':[0.01, 0.1, 1, 5, 10]} # 15번 iteration
grid_cv_lr = GridSearchCV(lr_clf ,param_grid=params , cv=3 , scoring='accuracy' , verbose=1 )
grid_cv_lr.fit(X_train_tfidf_vect , y_train)
print('Logistic Regression best C parameter :',grid_cv_lr.best_params_ )

# 최적 C 값으로 학습된 grid_cv로 예측 수행하고 정확도 평가. 
pred = grid_cv_lr.predict(X_test_tfidf_vect)
print('TF-IDF Vectorized Logistic Regression 의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test ,pred)))

Fitting 3 folds for each of 5 candidates, totalling 15 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
E:\anaconda\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
E:\anaconda\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/

Logistic Regression best C parameter : {'C': 10}
TF-IDF Vectorized Logistic Regression 의 예측 정확도는 0.788


E:\anaconda\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


# 사이킷런 Pipeline 사용 및 GridSearchCV와 결합

## Pipeline

In [28]:
from sklearn.pipeline import Pipeline

# TfidfVectorizer 객체를 tfidf_vect 객체명으로, LogisticRegression객체를 lr_clf 객체명으로 생성하는 Pipeline생성
pipeline = Pipeline([
    ('tfidf_vect', TfidfVectorizer(stop_words='english', max_df=300)),
    ('lr_clf', LogisticRegression(C=10))
])

# 별도의 TfidfVectorizer객체의 fit_transform( )과 LogisticRegression의 fit(), predict( )가 필요 없음. 
# pipeline의 fit( ) 과 predict( ) 만으로 한꺼번에 Feature Vectorization과 ML 학습/예측이 가능. 
pipeline.fit(X_train, y_train)
pred = pipeline.predict(X_test)
print('Pipeline을 통한 Logistic Regression 의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test ,pred)))

E:\anaconda\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Pipeline을 통한 Logistic Regression 의 예측 정확도는 0.788


## Pipeline + GridSearchCV

In [ ]:
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ('tfidf_vect', TfidfVectorizer(stop_words='english')),
    ('lr_clf', LogisticRegression())
])

# Pipeline에 기술된 각각의 객체 변수에 언더바(_)2개를 연달아 붙여 GridSearchCV에 사용될 
# 파라미터/하이퍼 파라미터 이름과 값을 설정. . 
params = { 'tfidf_vect__ngram_range': [(1,1), (1,2), (1,3)],
           'tfidf_vect__max_df': [100, 300, 700],
           'lr_clf__C': [1,5,10]
}

# GridSearchCV의 생성자에 Estimator가 아닌 Pipeline 객체 입력
grid_cv_pipe = GridSearchCV(pipeline, param_grid=params, cv=3 , scoring='accuracy',verbose=1)
grid_cv_pipe.fit(X_train , y_train)
print(grid_cv_pipe.best_params_ , grid_cv_pipe.best_score_)

pred = grid_cv_pipe.predict(X_test)
print('Pipeline을 통한 Logistic Regression 의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test ,pred)))

Fitting 3 folds for each of 27 candidates, totalling 81 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


너무 오래걸림